import os
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams
from matplotlib.ticker import FormatStrFormatter
import dask_searchcv as dcv
from distributed import Executor
from sklearn.model_selection import ShuffleSplit, GridSearchCV
from sklearn.externals.joblib import Parallel, parallel_backend, \
    register_parallel_backend
from distributed.joblib import DistributedBackend
%matplotlib inline

In [ ]:
from dalila.parameters_research import tune_parameters
from dalila.dictionary_learning import DictionaryLearning
from dalila.penalty import L1Penalty
from dalila.cv_splitting import MonteCarloBootstrap

In [ ]:
from utils import *

# Load data

In [ ]:
from scipy.io import loadmat
from unicodedata import normalize
filename = "/home/veronica/Desktop/UVM/mutation_signatures/datasets/breast_cancer_data.mat"
data = loadmat(filename, appendmat=False)
v = data["originalGenomes"]
types = data["types"]
l = len(types)
types_1 = [None] * l
for i in range(0, l):
    types_1[i] = normalize('NFKD', types[i][0][0]).encode('ascii','ignore')
data = v.T
types = np.asarray(types_1)

Remove "weak" mutations from the dataset

In [ ]:
res = remove_weak_mutations(data, 0.01)
X = res["mutational_catalogue"]
removed_cols = res["removed_cols"]

#  Extract mutational signatures

For each possible number of signatures it extracts the dictionary and the coefficients using Dictionary Learning and finding the best regularisation parameters in the grid using Cross-Validation. It performs the fit procedure in parallel, on the same machine, or distributing the jobs with dask. 

In [ ]:
results = []
ss = MonteCarloBootstrap(n_splits=3, test_size=0.2)
for k in range(2,10):
    estimator = DictionaryLearning(k=k,
                               dict_penalty=L1Penalty(1),
                               coeff_penalty=L1Penalty(1),
                               dict_normalization=1,
                               non_negativity="both")
    
    params = {'dict_penalty': estimator.dict_penalty.make_grid(0.01,0.1, 5),
              'coeff_penalty': estimator.dict_penalty.make_grid(0.001,0.01, 5)}

    gscv = GridSearchCV(estimator, params, cv=ss,
                         iid=True, refit=True, n_jobs=14,
                         verbose=1)

    #with parallel_backend('distributed',scheduler_host='10.251.61.227:8786'):
    gscv.fit(X)

    results.append(gscv)
    


In [ ]:
## Compute and plot BIC score

In [ ]:
BIC = []
for i in range(len(results)):
    estimator = results[i].best_estimator_
    BIC.append(- np.log(X.shape[0])*np.log(estimator.k) 
               + 2.3*np.log(estimator.objective_function_value()))

In [ ]:
fig, ax1 = plt.subplots(figsize=(5,5))
markers_on = [3]
ax1.plot(np.arange(2,9), -BICs[0:7], '-rD', markevery=markers_on, label="BICs")
ax1.set_xlabel("Number of mutational signatures")
ax1.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3)
plt.rcParams['axes.facecolor'] = (0.5, 0.294, 0.294, 0.3)
fig.tight_layout()
plt.show()

Select the best number given the plot

In [ ]:
C,D = results[3].best_estimator_.decomposition()

Re-insert the removed "weak" columns

In [ ]:
D = add_removed_cols(D, removed_cols)
D_ordered = ordering_for_types(D, types)

# Plot the resulting atoms

In [ ]:
for i in range(D.shape[0]):
    plot_atom(our_D[i,:])

# Analysis of the coefficients

In [ ]:
percentages = np.zeros_like(C)

for sample in range(C.shape[0]):
    total = np.sum(C[sample,:])
    if(total != 0):
        percentages[sample,:] = C[sample, :] / total 

print(percentages)

In [ ]:
frequencies = np.zeros(D.shape[0])
for atom in range(percentages.shape[1]):
    frequencies[atom]= len(np.nonzero(percentages[:,atom])[0])
plt.figure()
plt.hist(np.arange(D.shape[0]),weights=frequencies, bins=D.shape[0], 
         orientation="horizontal", alpha=0.5, histtype='bar', ec='black');
plt.show()